# C$50 Finance

In this guide we want to implement a website via which users can “register”, “login” “buy” and “sell” stocks, like below:

<img src="https://github.com/magnooj/CS50-finance/blob/main/static/1.png?raw=true" alt="Picture of dashboard">

## Background

If you’re not quite sure what it means to buy and sell stocks (i.e., shares of a company), head [here](https://www.investopedia.com/articles/basics/06/invest1000.asp) for a tutorial.

We’re about to implement C$50 Finance, a web app via which you can manage portfolios of stocks. Not only will this tool allow us to check real stocks’ actual prices and portfolios’ values, it will also let you buy and sell stocks by querying [IEX](https://iextrading.com/developer/) for stocks’ prices.

Indeed, IEX lets you download stock quotes via their API (application programming interface) using URLs like `https://cloud.iexapis.com/stable/stock/nflx/quote?token=API_KEY`. 

Before getting started on this project, we’ll need to register for an API key in order to be able to query IEX’s data. To do so, follow these steps:

- Visit iexcloud.io/cloud-login#/register/.
- Select the “Individual” account type, then enter your email address and a password, and click “Create account”.
- Once registered, scroll down to “Get started for free” and click “Select Start” to choose the free plan.
- Once you’ve confirmed your account via a confirmation email, visit (https://iexcloud.io/console/tokens).
- Copy the key that appears under the Token column (it should begin with `pk_`).
- In a terminal window execute:


In [ ]:
export API_KEY=value

where `value` is that (pasted) value, without any space immediately before or after the `=`. You also may wish to paste that value in a text document somewhere, in case you need it again later.

## Install requirements

This guide wrote for Windows Terminal and if you have another OS you may change it. 

Before we start, you should clone this [GitHub repo](https://github.com/magnooj/CS50-finance) and then install the dependencies.


In [ ]:
git clone https://github.com/magnooj/CS50-finance.git
cd CS50-fincance
pip install -r requirements.txt

## Through the files

Now, we are ready to run and test our project. By running `ls` you can see these files:

- [`app.py`](https://github.com/magnooj/CS50-finance/blob/main/app.py) : The APIs
- [`finance.db`](https://github.com/magnooj/CS50-finance/blob/main/finance.db) : Database of users and transactions
- [`helpers.py`](https://github.com/magnooj/CS50-finance/blob/main/helpers.py) : Some helpers algorithms
- [`requirements.txt`](https://github.com/magnooj/CS50-finance/blob/main/requirements.txt) : Required packages 
- [`/static/`](https://github.com/magnooj/CS50-finance/static/) : Favicon and CSS files
- [`/templates/`](https://github.com/magnooj/CS50-finance/templates/) : All the HTML files we needed to run this app that includes `jinja` codes in them.

### ***Flask API***

The first step in building APIs is to think about the data we want to handle, how we want to handle it and what output we want with our APIs. In our example, we want users can register, log in, log out and buy, sell and qout stocks; Finally, see the history of their transactions.

Let's tale a look at `app.py`:

In [ ]:
# We got some help from this youtube playlist: https://www.youtube.com/playlist?list=PLsJNRspv63XRbsJizNi55DE6FAjx7yK-x
import os

from cs50 import SQL
from flask import Flask, flash, redirect, render_template, request, session
from flask_session import Session
from tempfile import mkdtemp
from werkzeug.exceptions import default_exceptions, HTTPException, InternalServerError
from werkzeug.security import check_password_hash, generate_password_hash

from helpers import apology, login_required, lookup, usd

# Configure application
app = Flask(__name__)

# Ensure templates are auto-reloaded
app.config["TEMPLATES_AUTO_RELOAD"] = True


# Ensure responses aren't cached
@app.after_request
def after_request(response):
    response.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    response.headers["Expires"] = 0
    response.headers["Pragma"] = "no-cache"
    return response


# Custom filter
app.jinja_env.filters["usd"] = usd

# Configure session to use filesystem (instead of signed cookies)
app.config["SESSION_FILE_DIR"] = mkdtemp()
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

# Configure CS50 Library to use SQLite database
db = SQL("sqlite:///finance.db")

# Make sure API key is set
if not os.environ.get("API_KEY"):
    raise RuntimeError("API_KEY not set")


@app.route("/")
@login_required
def index():
    """Show portfolio of stocks"""
    user_id = session["user_id"]

    stocks = db.execute(
        "SELECT symbol, name, price, SUM(shares) as totalshares FROM transactions WHERE user_id = ? GROUP BY symbol", user_id)
    cash = db.execute("SELECT cash FROM users WHERE id = ?", user_id)[0]["cash"]

    total = cash

    for stock in stocks:
        total += stock["price"] * stock["totalshares"]

    return render_template("index.html", stocks=stocks, cash=cash, usd=usd, total=total)


@app.route("/buy", methods=["GET", "POST"])
@login_required
def buy():
    """Buy shares of stock"""
    if request.method == "POST":
        symbol = request.form.get("symbol").upper()
        item = lookup(symbol)

        if not symbol:
            return apology("Please enter a symbol!")
        elif not item:
            return apology("Invalid symbol!")

        try:
            shares = int(request.form.get("shares"))
        except:
            return apology("Shares must be an integer!")

        if not shares > 0:
            return apology("Shares must be positive integer!")

        user_id = session["user_id"]
        cash = db.execute("SELECT cash FROM users WHERE id = ?", user_id)[0]["cash"]

        item_name = item["name"]
        item_price = item["price"]
        total_price = item_price * shares

        if cash < total_price:
            return apology("Not enough money!")
        else:
            db.execute("UPDATE users SET cash = ? WHERE id = ?", cash - total_price, user_id)
            db.execute("INSERT INTO transactions (user_id, name, shares, price, type, symbol) VALUES (?, ?, ?, ?, ?, ?)",
                       user_id, item_name, shares, item_price, "buy", symbol)

        return redirect("/")
    else:
        return render_template("buy.html")


@app.route("/history")
@login_required
def history():
    """Show history of transactions"""
    user_id = session["user_id"]
    transactions = db.execute("SELECT type, symbol, price, shares, time FROM transactions WHERE user_id = ?", user_id)

    return render_template("history.html", transactions=transactions, usd=usd)


@app.route("/login", methods=["GET", "POST"])
def login():
    """Log user in"""

    # Forget any user_id
    session.clear()

    # User reached route via POST (as by submitting a form via POST)
    if request.method == "POST":

        # Ensure username was submitted
        if not request.form.get("username"):
            return apology("must provide username", 403)

        # Ensure password was submitted
        elif not request.form.get("password"):
            return apology("must provide password", 403)

        # Query database for username
        rows = db.execute("SELECT * FROM users WHERE username = ?", request.form.get("username"))

        # Ensure username exists and password is correct
        if len(rows) != 1 or not check_password_hash(rows[0]["hash"], request.form.get("password")):
            return apology("invalid username and/or password", 403)

        # Remember which user has logged in
        session["user_id"] = rows[0]["id"]

        # Redirect user to home page
        return redirect("/")

    # User reached route via GET (as by clicking a link or via redirect)
    else:
        return render_template("login.html")


@app.route("/logout")
def logout():
    """Log user out"""

    # Forget any user_id
    session.clear()

    # Redirect user to login form
    return redirect("/")


@app.route("/quote", methods=["GET", "POST"])
@login_required
def quote():
    """Get stock quote."""
    if request.method == "POST":
        symbol = request.form.get("symbol")

        if not symbol:
            return apology("Please enter a sumbol!")

        item = lookup(symbol)

        if not item:
            return apology("Invalid symbol!")

        return render_template("quoted.html", item=item, usd=usd)
    else:
        return render_template("quote.html")


@app.route("/register", methods=["GET", "POST"])
def register():
    """Register user"""
    if request.method == "POST":
        username = request.form.get("username")
        password = request.form.get("password")
        confirmation = request.form.get("confirmation")

        if not username:
            return apology("Username is required!")
        elif not password:
            return apology("Password is required!")
        elif not confirmation:
            return apology("Confirm your password, please!")

        if password != confirmation:
            return apology("Passwords did not match!")

        hash = generate_password_hash(password)

        try:
            db.execute("INSERT INTO users (username, hash) VALUES(?, ?)", username, hash)
            return redirect("/")
        except:
            return apology("Username has alredy been registered!")
    else:
        return render_template("register.html")


@app.route("/sell", methods=["GET", "POST"])
@login_required
def sell():
    """Sell shares of stock"""
    user_id = session["user_id"]

    if request.method == "POST":
        symbol = request.form.get("symbol")
        shares = int(request.form.get("shares"))

        if shares <= 0:
            return apology("Shares must be a positive number!")

        item_price = lookup(symbol)["price"]
        item_name = lookup(symbol)["name"]
        price = shares * item_price

        shares_owned = db.execute("SELECT SUM(shares) as tshare FROM transactions WHERE user_id = ? AND symbol = ?", user_id, symbol)[
            0]["tshare"]
        if shares_owned < shares:
            return apology("You don't have enough shares!")

        current_cash = db.execute("SELECT cash FROM users WHERE id = ?", user_id)[0]["cash"]
        db.execute("UPDATE users SET cash = ? WHERE id = ?", current_cash + price, user_id)
        db.execute("INSERT INTO transactions (user_id, name, shares, price, type, symbol) VALUES (?, ?, ?, ?, ?, ?)",
                   user_id, item_name, -shares, item_price, "sell", symbol)

        return redirect("/")
    else:
        symbols = db.execute("SELECT symbol FROM transactions WHERE user_id = ? GROUP BY symbol", user_id)
        return render_template("sell.html", symbols=symbols)


def errorhandler(e):
    """Handle error"""
    if not isinstance(e, HTTPException):
        e = InternalServerError()
    return apology(e.name, e.code)


# Listen for errors
for code in default_exceptions:
    app.errorhandler(code)(errorhandler)


The main HTML file in our app is `layout.html`. We created a template that other HTML files cand extend that. Let's see inside of it:

In [ ]:
<!DOCTYPE html>

<html lang="en">

    <head>

        <meta charset="utf-8">
        <meta name="viewport" content="initial-scale=1, width=device-width">

        <!-- http://getbootstrap.com/docs/4.5/ -->
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/css/bootstrap.min.css" integrity="sha384-TX8t27EcRE3e/ihU7zmQxVncDAy5uIKz4rEkgIXeMed4M0jlfIDPvg6uqKI2xXr2" crossorigin="anonymous">

        <!-- https://favicon.io/emoji-favicons/money-mouth-face/ -->
        <link href="/static/favicon.ico" rel="icon">

        <link href="/static/styles.css" rel="stylesheet">

        <!-- http://getbootstrap.com/docs/4.5/ -->
        <script src="https://code.jquery.com/jquery-3.5.1.slim.min.js" integrity="sha384-DfXdz2htPH0lsSSs5nCTpuj/zy4C+OGpamoFVy38MVBnE+IbbVYUew+OrCXaRkfj" crossorigin="anonymous"></script>
        <script src="https://cdn.jsdelivr.net/npm/bootstrap@4.5.3/dist/js/bootstrap.bundle.min.js" integrity="sha384-ho+j7jyWK8fNQe+A12Hb8AhRq26LrZ/JpcUGGOn+Y7RsweNrtN/tE3MoK7ZeZDyx" crossorigin="anonymous"></script>

        <title>C$50 Finance: {% block title %}{% endblock %}</title>

    </head>

    <body>

        <nav class="navbar navbar-expand-md navbar-light bg-light border">
            <a class="navbar-brand" href="/"><span class="blue">C</span><span class="red">$</span><span class="yellow">5</span><span class="green">0</span> <span class="red">Finance</span></a>
            <button aria-controls="navbar" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbar" data-toggle="collapse" type="button">
                <span class="navbar-toggler-icon"></span>
            </button>
            <div class="collapse navbar-collapse" id="navbar">
                {% if session.user_id %}
                    <ul class="navbar-nav mr-auto mt-2">
                        <li class="nav-item"><a class="nav-link" href="/quote">Quote</a></li>
                        <li class="nav-item"><a class="nav-link" href="/buy">Buy</a></li>
                        <li class="nav-item"><a class="nav-link" href="/sell">Sell</a></li>
                        <li class="nav-item"><a class="nav-link" href="/history">History</a></li>
                    </ul>
                    <ul class="navbar-nav ml-auto mt-2">
                        <li class="nav-item"><a class="nav-link" href="/logout">Log Out</a></li>
                    </ul>
                {% else %}
                    <ul class="navbar-nav ml-auto mt-2">
                        <li class="nav-item"><a class="nav-link" href="/register">Register</a></li>
                        <li class="nav-item"><a class="nav-link" href="/login">Log In</a></li>
                    </ul>
                {% endif %}
            </div>
        </nav>

        {% if get_flashed_messages() %}
            <header>
                <div class="alert alert-primary border text-center" role="alert">
                    {{ get_flashed_messages() | join(" ") }}
                </div>
            </header>
        {% endif %}

        <main class="container p-5">
            {% block main %}{% endblock %}
        </main>

        <footer class="small text-center text-muted">
            Data provided for free by <a href="https://iextrading.com/developer">IEX</a>. View <a href="https://iextrading.com/api-exhibit-a/">IEX’s Terms of Use</a>.
        </footer>

    </body>

</html>


In this example, we create Flask eight routs so that we can serve HTTP traffic on that route.

- `/` or `index` : Is the homepage of our app. If user loged in, it display the user’s current cash balance along with a grand total (i.e., stocks’ total value plus cash). But, if user didn.t log in, it displays the login page.
- `register` : It has a form that user can register by filling it.
- `buy` : In this route, users can input a stock’s symbol and buy some shares.
- `sell` : In this page, users can `SELECT` from theis stocks’ symbol and sell their shares.
- `qoute` : Users can lookup the price each share in a stock’s symbol.
- `history` : It displays an HTML table summarizing all of a user’s transactions ever, listing row by row each and every buy and every sell.
- `login` and `logout` : These routes start and terminate user’s session.

Of course there is some files like `apology.html` that displays the error to the user. You can check other files.

Let's see one of HTML files. I think `index` page is good to look at it:

In [ ]:
{% extends "layout.html" %}

{% block title %}
    Dashboard
{% endblock %}

{% block main %}
    <table class="table table-striped">
        <thead>
            <tr>
                <th>Symbol</th>
                <th>Name</th>
                <th>Shares</th>
                <th>Price</th>
                <th>TOTAL</th>
            </tr>
        </thead>
        <!-- Create a table  by jinja that shows the stocks -->
        <tbody>
            {% for stock in stocks %}
                {% if stock["totalshares"] != 0 %}
                    <tr>
                        <td> {{ stock["symbol"] }}</td>
                        <td> {{ stock["name"] }}</td>
                        <td> {{ stock["totalshares"] }}</td>
                        <td> {{ usd(stock["price"]) }}</td>
                        <td> {{ usd(stock["totalshares"] * stock["price"]) }}</td>
                    </tr>
                {% endif %}
            {% endfor %}
            <tr>
                <td>CASH</td>
                <td colspan="3"></td>
                <td>{{ usd(cash) }}</td>
            </tr>
        </tbody>
        <tfoot>
            <td colspan="4"></td>
            <td><strong>{{ usd(total) }}</strong></td>
        </tfoot>
    </table>
{% endblock %}

Now, We cheked our files and sqw how our app is working. To run the app, when you are in `CS50-finance` directory, enter `flask run` in the terminal.

I hope you enjoyed how to stocks' exchange web application using flask. if you have any comments please do not hesitate to send me an [e-mail](mailto:magnooj@gmail.com).

Regards,

Ali Ganjizadeh